# 🌐 Laboratório: Busca Semântica com APIs em Nuvem

## O que vamos aprender?

Neste notebook, você vai construir um **sistema de busca inteligente** usando modelos de embeddings hospedados na **nuvem** (OpenAI ou Google) e comparar diferentes **métricas de similaridade**.

### Diferença entre Busca Tradicional vs. Busca Semântica

**Busca Tradicional (palavras-chave):**

```text
Query: "hardware celular"
Resultado: Só encontra documentos com as palavras exatas "hardware" e "celular"
```

**Busca Semântica (significado):**

```text
Query: "hardware celular"
Resultado: Encontra "iPhone", "smartphone", "processador mobile" 
           mesmo sem as palavras exatas!
```

### Objetivos deste Lab

1. ✅ Criar embeddings usando OpenAI API
2. ✅ Indexar documentos com FAISS
3. ✅ Comparar **Distância L2 (Euclidiana)** vs **Similaridade de Cosseno**
4. ✅ Visualizar e interpretar resultados
5. ✅ Entender quando usar cada métrica

### Por que usar APIs em Nuvem?

✅ **Qualidade superior** - Modelos treinados com bilhões de documentos  
✅ **Sem instalação** - Não precisa configurar nada localmente  
✅ **Sempre atualizado** - Melhorias automáticas pela empresa  
⚠️ **Custo** - Paga por uso (mas há tiers gratuitos)  
⚠️ **Privacidade** - Dados trafegam pela internet

💡 **Comparação:** Se você precisa de máxima qualidade e tem orçamento, use nuvem. Se precisa de privacidade/gratuidade, use Ollama local (veja `lab_2.2`).

## 🛠️ Ferramentas que Vamos Usar

### 1. LangChain 🦜🔗
**O que é:** Framework Python que simplifica a integração com APIs de IA  
**Por que usar:** Código padronizado que funciona com OpenAI, Google, Anthropic, etc.  
**Analogia:** É como usar um adaptador universal - você troca de API mudando 1 linha de código

### 2. OpenAI Embeddings ☁️
**O que é:** API da empresa criadora do ChatGPT para gerar embeddings  
**Modelo:** `text-embedding-3-small` - 1536 dimensões  
**Qualidade:** ⭐⭐⭐⭐⭐ Estado da arte (SOTA)  
**Custo:** ~$0.02 por 1 milhão de tokens (muito barato)

**Alternativa:** Google Generative AI Embeddings
- Modelo: `text-embedding-004` - 768 dimensões
- Também de alta qualidade e com tier gratuito generoso

### 3. FAISS 🚀
**O que é:** Biblioteca do Facebook/Meta para busca vetorial ultrarrápida  
**Por que usar:** Milhões de buscas por segundo, roda em memória  
**Ideal para:** Protótipos, aplicações com poucos documentos (< 100k)

**Fluxo completo:**
```
Texto → OpenAI API → Vetor [1536 números] → FAISS → Busca instantânea
```

### Comparação: Nuvem vs. Local

| Aspecto | OpenAI (Nuvem) | Ollama (Local) |
|---------|----------------|----------------|
| **Qualidade** | ⭐⭐⭐⭐⭐ | ⭐⭐⭐ |
| **Velocidade** | ~100ms (latência rede) | ~10ms |
| **Custo** | $0.02/1M tokens | Gratuito |
| **Privacidade** | Dados na nuvem | 100% local |
| **Setup** | Só precisa API key | Instalar Ollama |

## 🎯 Cenário do Experimento

Vamos criar um **"mini Google"** com documentos de 3 categorias completamente diferentes:

| Categoria | Documentos |
|-----------|------------|
| 🖥️ **Tecnologia** | iPhone 15, RTX 4090 |
| 🍰 **Culinária** | Receita de bolo, Lasanha |
| ⚽ **Esportes** | Gol de futebol |

### O Desafio

Faremos uma pergunta que **não contém palavras exatas** dos documentos:

**Pergunta:** "Quero sugestões de hardware para computador ou celular"

**Expectativa:**
- ✅ Deve retornar: iPhone e RTX 4090 (são hardware!)
- ❌ NÃO deve retornar: Bolo, Lasanha, Gol (não são hardware)

### Por que isso é difícil?

Um sistema de busca tradicional (Ctrl+F) falharia porque:
- A palavra "iPhone" não aparece na pergunta
- A palavra "RTX 4090" não aparece na pergunta
- Mas a API da OpenAI **entende** que ambos são hardware!

💡 **Isso é Inteligência Artificial em ação!** O modelo GPT aprendeu que "iPhone = celular = hardware" e "RTX 4090 = placa de vídeo = hardware" durante seu treinamento massivo.

In [1]:
import os
import requests
from pathlib import Path
from dotenv import load_dotenv
from langchain_community.vectorstores import FAISS

# Se for usar OpenAI
from langchain_openai import OpenAIEmbeddings as CloudEmbeddings

# Se for usar Google Generative AI
# from langchain_google_genai import GoogleGenerativeAIEmbeddings as CloudEmbeddings

## 🔌 Imports e Configuração

Bibliotecas que vamos usar:
- `os`: Para ler variáveis de ambiente (API keys)
- `Path` e `load_dotenv`: Para carregar configurações do `.env`
- `FAISS`: Banco de dados vetorial
- `OpenAIEmbeddings`: Modelo de embeddings da OpenAI

⚠️ **Importante:** Você precisa ter uma API key da OpenAI ou do Google no arquivo `.env`:

```ini
OPENAI_API_KEY=sk-proj-...
# GOOGLE_API_KEY=AIza...
```



**Como obter API key:**
1. Crie conta em [platform.openai.com](https://platform.openai.com)
2. Vá em "API Keys" e clique "Create new secret key"
3. Copie e cole no arquivo `.env`

In [2]:
# 2) Configuração e carregamento do .env (simplificado)
env_path = Path.cwd().joinpath('..', '..', '.env').resolve()
if env_path.exists():
    load_dotenv(env_path)
    print(f'🔎 .env carregado -> {env_path.resolve()}')
else:
    print('⚠️  .env não encontrado. Defina as variáveis de ambiente manualmente.')


🔎 .env carregado -> E:\01-projetos\11-work\11.34-engenharia-vetorial\.env


## 🔐 Carregando Variáveis de Ambiente

O arquivo `.env` contém informações sensíveis:
- `OPENAI_API_KEY`: Sua chave de API da OpenAI
- `GOOGLE_API_KEY`: (Opcional) Sua chave do Google AI

**Por que usar `.env`?**
- ✅ Segurança: Não expor chaves no código
- ✅ Portabilidade: Funciona em dev e produção
- ✅ Controle: Fácil trocar entre contas

**Estrutura do `.env`:**
```bash
OPENAI_API_KEY=sk-proj-abc123...
# GOOGLE_API_KEY=AIza...  # Descomente se usar Google
```

## 📄 Passo 1: Preparando os Documentos

Aqui temos nosso **dataset de teste** — 5 documentos de categorias bem distintas.

**Por que categorias diferentes?**
Queremos testar se a IA consegue distinguir contextos. Um bom modelo de embedding deve:
- Colocar "iPhone" e "RTX 4090" próximos (ambos são tecnologia/hardware)
- Colocar "bolo" e "lasanha" próximos (ambos são culinária)
- Manter "gol" distante de receitas e hardware (esporte é outro contexto)

### Estrutura dos Dados

```python
meus_textos = [
    "Documento sobre tecnologia...",  # Categoria: Tech
    "Documento sobre culinária...",   # Categoria: Food
    "Documento sobre esporte...",     # Categoria: Sport
]
```

💡 **Conceito importante:** Esses textos são chamados de **corpus** (conjunto de documentos que queremos buscar).

### Em Produção

Em uma aplicação real, esses textos viriam de:
- 📄 PDFs extraídos
- 🌐 Páginas web (scraping)
- 💬 Histórico de conversas
- 📊 Documentação técnica

In [3]:

meus_textos = [
    "O novo iPhone 15 tem uma lente periscópica incrível.",    # Tecnologia
    "Para fazer um bolo macio, bata as claras em neve.",       # Culinária
    "O atacante chutou a bola no ângulo e foi gol.",           # Esporte
    "A placa de vídeo RTX 4090 roda jogos em 4K.",             # Tecnologia
    "Receita de lasanha à bolonhesa com muito queijo."         # Culinária
]

## 🧠 Passo 2: Inicializando o Modelo de Embeddings

Este é o **"cérebro"** que transforma texto em vetores numéricos via API!

### O que são Embeddings?

**Embedding** = Representação numérica de um texto

Exemplo com OpenAI:

```text
Texto: "gato"
↓ (envia para API da OpenAI)
Embedding: [0.12, -0.34, 0.56, 0.89, ..., 0.23]
           ↑ vetor com 1536 números!
```

### Como funciona?

1. Seu texto é enviado para os servidores da OpenAI.
2. O modelo GPT (treinado em trilhões de palavras) processa os textos.
3. Retorna um vetor que "captura" o significado.
4. Textos similares têm vetores próximos.

**Visualização conceitual:**

```text
Espaço vetorial (imagine em 3D, mas é 1536D!)

  gato • ← próximo de → • felino
  
  carro • ← DISTANTE de → • felino
```

### Modelos Disponíveis

**OpenAI:**
| Modelo | Dimensões | Custo/1M tokens | Qualidade |
|--------|-----------|-----------------|-----------|
| `text-embedding-3-small` | 1536 | $0.02 | ⭐⭐⭐⭐⭐ |
| `text-embedding-3-large` | 3072 | $0.13 | ⭐⭐⭐⭐⭐ |
| `text-embedding-ada-002` | 1536 | $0.10 | ⭐⭐⭐⭐ (legado) |

**Google AI:**
| Modelo | Dimensões | Custo/1M tokens | Qualidade |
|--------|-----------|-------|-----------|
| `text-embedding-004` | 768 | Grátis (limite) | ⭐⭐⭐⭐⭐ |
| `gemini-embedding-001` | 3072 | Grátis (limite), depois $0.15  | ⭐⭐⭐⭐⭐ |

### Qual escolher?

- **Protótipo/teste:** `text-embedding-3-small` (OpenAI) - barato e excelente
- **Máxima qualidade:** `text-embedding-3-large` (OpenAI)
- **Grátis:** `text-embedding-004` (Google) - tier gratuito generoso

💡 **Dica:** O modelo `-small` da OpenAI já é melhor que a maioria dos modelos open-source!

In [4]:
# Se for usar OpenAI
embeddings = CloudEmbeddings(model="text-embedding-3-small")

# Se for usar Google Generative AI
# embeddings = CloudEmbeddings(model="models/text-embedding-004")

## 🗄️ Passo 3: Criando o Banco de Dados Vetorial (Indexação)

Aqui acontece a **mágica**! Vamos transformar todos os textos em vetores e armazená-los no FAISS.

### O que acontece nesta linha de código?

```python
vector_store = FAISS.from_texts(meus_textos, embeddings)
```

**Passo a passo interno:**

1. **Para cada texto**, LangChain faz uma chamada à API:
   ```text
   "O novo iPhone 15..." 
   → 🌐 API OpenAI 
   → [0.2, -0.5, 0.8, ..., 0.15] (1536 números)
   
   "Para fazer um bolo..." 
   → 🌐 API OpenAI 
   → [0.1, 0.3, -0.4, ..., -0.22] (1536 números)
   ```

2. **FAISS armazena** todos esses vetores em memória:
   ```text
   Índice FAISS (RAM):
   [0] → [0.2, -0.5, ...] (iPhone)
   [1] → [0.1, 0.3, ...] (bolo)
   [2] → [-0.3, 0.7, ...] (gol)
   [3] → [0.25, -0.48, ...] (RTX 4090)
   [4] → [0.12, 0.28, ...] (lasanha)
   ```

3. **Pronto!** Agora podemos fazer buscas instantâneas

### Custo desta operação

Com OpenAI `text-embedding-3-small`:
```text
5 documentos × ~20 tokens cada = 100 tokens
Custo: 100 / 1.000.000 × $0.02 = $0.000002 (negligível!)
```

### Analogia

Imagine uma biblioteca com milhões de livros:
- **Sem índice:** Ler cada livro até achar o certo 😰
- **Com FAISS:** Sistema Dewey Decimal turbinado que te leva direto ao livro! 📚✨

### 📏 Métricas de Similaridade: L2 vs Cosseno

FAISS suporta diferentes formas de medir "proximidade" entre vetores. Vamos criar **dois índices** e comparar:

#### 1. Distância L2 (Euclidiana)

**O que mede:** A distância "em linha reta" entre dois pontos no espaço vetorial.

**Fórmula matemática:**
$$d_{L2}(v_1, v_2) = \sqrt{\sum_{i=1}^{n}(v_{1i} - v_{2i})^2}$$

**Analogia:** Distância entre duas cidades no mapa (linha reta).

**Características:**
- ✅ Simples e intuitiva
- ✅ Considera magnitude dos vetores
- ⚠️ Sensível à escala (vetores maiores têm distâncias maiores)
- **Interpretação:** Menor = mais similar (0 = idêntico)


#### 2. Similaridade Cosseno

**O que mede:** O ângulo entre dois vetores (ignora magnitude).

**Fórmula matemática:**
$$\text{cosine}(v_1, v_2) = \frac{v_1 \cdot v_2}{\|v_1\| \times \|v_2\|}$$

**Analogia:** Direção de duas setas, independente do tamanho.

**Características:**
- ✅ Normalizado entre -1 e 1
- ✅ Invariante à magnitude (só direção importa)
- ✅ Muito usado em NLP e embeddings
- **Interpretação:** Maior (cosine) = mais similar (1 = idêntico, 0 = perpendicular, -1 = oposto)

Não confundir maioridade do cosseno com a abertura do ângulo!

#### Quando usar cada uma?

| Aspecto | L2 (Euclidiana) | Cosseno |
|---------|-----------------|---------|
| **Sensível à magnitude** | ✅ Sim | ❌ Não |
| **Normalização necessária** | ❌ Não | ✅ Sim (FAISS faz automaticamente) |
| **Uso comum em NLP** | 🟡 Médio | 🟢 Alto |
| **Embeddings OpenAI** | 🟢 Funciona bem | 🟢 Funciona bem (recomendado) |
| **Interpretação** | Distância (menor = melhor) | Similaridade (maior = melhor) |

**💡 Curiosidade:** OpenAI recomenda similaridade de cosseno para seus embeddings, mas ambas funcionam bem na prática!

### FAISS: Tipos de Índice

Vamos criar dois índices diferentes:

1. **IndexFlatL2** - Distância Euclidiana
   ```python
   FAISS.from_texts(..., distance_strategy="EUCLIDEAN_DISTANCE")
   ```

2. **IndexFlatIP** - Inner Product (Produto Interno)
   ```python
   FAISS.from_texts(..., distance_strategy="MAX_INNER_PRODUCT")
   ```
   - Para vetores normalizados, IP é equivalente a similaridade de cosseno
   - OpenAI já normaliza os embeddings automaticamente!



In [5]:
from langchain_community.vectorstores.utils import DistanceStrategy

# Criar índice com Distância L2 (Euclidiana)
print("📊 Criando índice com Distância L2 (Euclidiana)...")
vector_store_l2 = FAISS.from_texts(
    meus_textos, 
    embeddings,
    distance_strategy=DistanceStrategy.EUCLIDEAN_DISTANCE
)
print("✅ Índice L2 criado!\n")

# Criar índice com Similaridade de Cosseno (Inner Product para vetores normalizados)
print("📊 Criando índice com Similaridade de Cosseno...")
vector_store_cosine = FAISS.from_texts(
    meus_textos, 
    embeddings,
    distance_strategy=DistanceStrategy.MAX_INNER_PRODUCT
)
print("✅ Índice Cosseno criado!\n")

print("🎯 Dois índices prontos para comparação!")

📊 Criando índice com Distância L2 (Euclidiana)...
✅ Índice L2 criado!

📊 Criando índice com Similaridade de Cosseno...
✅ Índice Cosseno criado!

🎯 Dois índices prontos para comparação!


---

## 🚀 Hora de Testar! Vamos Fazer Buscas

Configuração concluída! ✅  
Agora vamos usar nosso sistema de busca semântica com a qualidade da OpenAI.

**O que fizemos até agora:**
1. ✅ Carregamos a API key da OpenAI
2. ✅ Criamos um modelo de embeddings (1536 dimensões)
3. ✅ Indexamos 5 documentos no FAISS 
4. ✅ FAISS está pronto para buscas instantâneas

**Próximo passo:** Fazer perguntas e ver a IA encontrar as respostas! 🎯

💰 **Custo até agora:** ~$0.000002 (menos que um centavo de centavo!)

## 🔍 Passo 4: Definindo a Pergunta (Query)

Aqui está o **teste de fogo** para a API da OpenAI!

### A Pergunta

```python
"Quero sugestões de hardware para computador ou celular"
```

### Por que esta pergunta é desafiadora?

**Palavras que NÃO aparecem na pergunta:**
- ❌ "iPhone"
- ❌ "RTX"
- ❌ "4090"
- ❌ "placa de vídeo"
- ❌ "lente periscópica"

**Palavras que SIM aparecem:**
- ✅ "hardware"
- ✅ "computador"
- ✅ "celular"

### O Desafio

Um sistema de busca tradicional (Ctrl+F) **falharia** porque:

```text
Busca por "hardware" → Não encontra nada (palavra não está nos docs)
Busca por "celular" → Não encontra "iPhone" (palavra diferente)
Busca por "computador" → Não encontra "RTX 4090" (palavra diferente)
```

### A Inteligência da OpenAI

A API vai **entender** que:

```text
"celular" ≈ "iPhone" ≈ "smartphone" ≈ "telefone móvel"
"computador" ≈ "PC" ≈ "placa de vídeo" ≈ "hardware desktop"
"hardware" ≈ "equipamento" ≈ "dispositivo eletrônico"
```

💡 **Isso é aprendizado semântico!** O modelo GPT foi treinado em uma quantidade massiva de texto da internet e aprendeu relações complexas entre conceitos.

### O que acontece nos bastidores?

1. Sua pergunta é enviada para a API da OpenAI
2. GPT transforma em vetor: `[0.22, -0.48, 0.79, ..., 0.31]`
3. FAISS compara com todos os vetores armazenados
4. Retorna os mais próximos (mesmo sem palavras iguais!)



In [6]:
query = "Quero sugestões de hardware para computador ou celular"

## 🎯 Passo 5: Comparando Métricas de Similaridade

Agora vamos **comparar** os resultados usando as duas métricas diferentes!

### O que significa `k=2`?

```python
vector_store.similarity_search(pergunta, k=2)
```

O parâmetro `k` define quantos resultados queremos:
- `k=1` → Retorna apenas o documento mais similar
- `k=2` → Retorna os 2 documentos mais similares
- `k=5` → Retorna os 5 documentos mais similares

### Fluxo de busca com cada métrica

#### 🔵 Distância L2 (Euclidiana)

1. **Sua pergunta vira um vetor (via API OpenAI):**
   ```text
   "Quero sugestões de hardware..." 
   → 🌐 API call
   → [0.22, -0.48, 0.79, ..., 0.31] (1536 números)
   ```

2. **FAISS calcula distâncias L2 (local, sem API):**
   ```text
   d_L2(query, iPhone) = 0.45      ← Menor distância = mais próximo!
   d_L2(query, bolo) = 2.89        ← Distante
   d_L2(query, gol) = 3.12         ← Muito distante
   d_L2(query, RTX 4090) = 0.52    ← Próximo!
   d_L2(query, lasanha) = 2.76     ← Distante
   ```

3. **Ordena por distância (menor = melhor):**
   ```text
   1º lugar: iPhone (0.45) ← Menor distância!
   2º lugar: RTX 4090 (0.52)
   ...
   ```

#### 🟢 Similaridade de Cosseno

1. **Mesmo vetor da query** (reutilizado do cache)

2. **FAISS calcula produto interno (= cosseno para vetores normalizados):**
   ```text
   cosine(query, iPhone) = 0.95      ← Alto = muito similar!
   cosine(query, bolo) = 0.23        ← Baixo = pouco similar
   cosine(query, gol) = 0.15         ← Muito baixo
   cosine(query, RTX 4090) = 0.92    ← Alto = similar!
   cosine(query, lasanha) = 0.28     ← Baixo
   ```

3. **Calcula o ângulo correspondente:**
   ```text
   ângulo = arccos(cosine)
   
   ângulo(query, iPhone) = arccos(0.95) ≈ 18°     ← Pequeno ângulo!
   ângulo(query, bolo) = arccos(0.23) ≈ 77°       ← Grande ângulo
   ângulo(query, RTX) = arccos(0.92) ≈ 23°        ← Pequeno ângulo!
   ```

4. **Ordena por similaridade (maior = melhor) ou por ângulo (menor = melhor):**
   ```text
   1º lugar: iPhone (0.95 / 18°) ← Maior similaridade / Menor ângulo!
   2º lugar: RTX 4090 (0.92 / 23°)
   ...
   ```

### Diferenças na prática

| Aspecto | L2 (Euclidiana) | Cosseno | 
|---------|-----------------|---------|
| **Métrica** | Distância | Fator Similaridade ou Ângulo | 
| **Escala** | 0 a ∞ (menor = melhor) | -1 a 1 (maior = melhor) ou 0° a 180° (menor = melhor) |
| **Sensibilidade** | Magnitude + direção | Apenas direção | 
| **Normalização** | Não requer | Automática no FAISS | 
| **Uso típico** | Dados não-normalizados | Embeddings de texto | 
| **Interpretação** | Distância geométrica | Produto escalar | 

### O que esperar?

✅ **Ambas devem retornar:** iPhone e RTX 4090 (são hardware!)  
✅ **Ângulos pequenos (<30°)** para documentos relevantes  
❌ **Nenhuma deve retornar:** Bolo, Lasanha, Gol (não são hardware)  
❌ **Ângulos grandes (>60°)** para documentos irrelevantes  

**Diferença esperada:** A **ordem** pode variar ligeiramente, mas os top-k geralmente são os mesmos!

**Por quê?** OpenAI já normaliza os embeddings, então L2 e Cosseno tendem a concordar na prática.

### Visualização conceitual (2D, mas é 1536D!)

![calculo similaridade](similaridades.png)

Vamos executar e comparar! 👇

💰 **Custo:** ~$0.000001 por query (uma busca no cache, sem nova API call)

In [7]:
import numpy as np
import pandas as pd

print("="*80)
print("🔍 COMPARAÇÃO DE MÉTRICAS DE SIMILARIDADE - TOP 2 RESULTADOS")
print("="*80)
print(f"\n📝 Pergunta: '{query}'\n")

# Buscar resultados com ambas métricas
resultados_l2 = vector_store_l2.similarity_search_with_score(query, k=2)
resultados_cosine = vector_store_cosine.similarity_search_with_score(query, k=2)

# Preparar dados para tabela comparativa
dados_tabela = []

# Processar resultados L2
for i, (doc, score) in enumerate(resultados_l2, 1):
    dados_tabela.append({
        'Rank': i,
        'Métrica': '🔵 L2 (Euclidiana)',
        'Score': f"{score:.4f}",
        'Interpretação': 'Menor = melhor',
        'Ângulo (°)': '-',
        'Documento': doc.page_content[:60] + "..." if len(doc.page_content) > 60 else doc.page_content
    })

# Processar resultados Cosseno
for i, (doc, score) in enumerate(resultados_cosine, 1):
    # Calcular ângulo
    similarity_clamped = np.clip(score, -1.0, 1.0)
    angulo_radianos = np.arccos(similarity_clamped)
    angulo_graus = np.degrees(angulo_radianos)
    
    dados_tabela.append({
        'Rank': i,
        'Métrica': '🟢 Cosseno',
        'Score': f"{score:.4f}",
        'Interpretação': 'Maior = melhor',
        'Ângulo (°)': f"{angulo_graus:.2f}",
        'Documento': doc.page_content[:60] + "..." if len(doc.page_content) > 60 else doc.page_content
    })

# Criar DataFrame
df_resultados = pd.DataFrame(dados_tabela)

# Criar tabela comparativa lado a lado
print("\n📊 COMPARAÇÃO LADO A LADO:\n")

# Preparar dados para tabela comparativa final
dados_lado_a_lado = []

for i in range(2):
    doc_l2, score_l2 = resultados_l2[i]
    doc_cosine, score_cosine = resultados_cosine[i]
    
    # Calcular ângulo para o cosseno
    similarity_clamped = np.clip(score_cosine, -1.0, 1.0)
    angulo_graus = np.degrees(np.arccos(similarity_clamped))
    
    dados_lado_a_lado.append({
        'Rank': i + 1,
        'Doc L2': doc_l2.page_content[:40] + "...",
        'Score L2': f"{score_l2:.4f}",
        # 'Doc Cosseno': doc_cosine.page_content[:40] + "...",
        'Score Cosseno': f"{score_cosine:.4f}",
        'Ângulo (°)': f"{angulo_graus:.2f}"
    })

df_lado_a_lado = pd.DataFrame(dados_lado_a_lado)
try:
    print(df_lado_a_lado.to_markdown(index=False))
except ImportError:
    print(df_lado_a_lado.to_string(index=False))

print("\n" + "="*80)
print("\n💡 INTERPRETAÇÃO:")
print("  • L2: Menor score = maior similaridade")
print("  • Cosseno: Maior score = maior similaridade")
print("  • Ângulo: Menor ângulo = maior similaridade (0° = idêntico)")

# Verificar concordância
if resultados_l2[0][0].page_content == resultados_cosine[0][0].page_content:
    print("\n✅ Ambas métricas concordam no documento mais relevante!")
else:
    print("\n⚠️  Métricas divergem - isso pode indicar diferenças na normalização")

print("="*80)

🔍 COMPARAÇÃO DE MÉTRICAS DE SIMILARIDADE - TOP 2 RESULTADOS

📝 Pergunta: 'Quero sugestões de hardware para computador ou celular'


📊 COMPARAÇÃO LADO A LADO:

 Rank                                      Doc L2 Score L2 Score Cosseno Ângulo (°)
    1 A placa de vídeo RTX 4090 roda jogos em ...   1.4026        0.2983      72.65
    2 O novo iPhone 15 tem uma lente periscópi...   1.4345        0.2828      73.57


💡 INTERPRETAÇÃO:
  • L2: Menor score = maior similaridade
  • Cosseno: Maior score = maior similaridade
  • Ângulo: Menor ângulo = maior similaridade (0° = idêntico)

✅ Ambas métricas concordam no documento mais relevante!


## 📊 Análise Comparativa dos Resultados

### O que observar?

Quando você executar a célula acima, observe:

1. **Os documentos retornados são os mesmos em ambas métricas?**
   - ✅ Esperado: Sim, geralmente os top-k são idênticos
   - 🤔 Se diferentes: Pode indicar que magnitude dos vetores importa

2. **A ordem é a mesma?**
   - ✅ Geralmente sim, pois OpenAI normaliza embeddings
   - 🤔 Pequenas variações são normais

3. **Os scores fazem sentido?**
   - **L2:** Valores menores (0.3-1.5 = muito relevante, >3.0 = pouco relevante)
   - **Cosseno:** Valores maiores (>0.9 = muito relevante, <0.5 = pouco relevante)

### Por que as métricas concordam?

**Embeddings da OpenAI são normalizados:**
- Todos os vetores têm magnitude ~1.0
- Quando magnitude é constante, L2 e Cosseno são equivalentes
- Fórmula: Para vetores unitários, $d_{L2}^2 \approx 2(1 - \text{cosine})$

**Visualização matemática:**

Se $\|v_1\| = \|v_2\| = 1$ (vetores normalizados):

$$
\begin{align}
d_{L2}(v_1, v_2)^2 &= \|v_1 - v_2\|^2 \\
&= \|v_1\|^2 + \|v_2\|^2 - 2(v_1 \cdot v_2) \\
&= 1 + 1 - 2(v_1 \cdot v_2) \\
&= 2(1 - \text{cosine}(v_1, v_2))
\end{align}
$$

Lembre-se que a **norma (magnitude)** de um vetor é:

$$\|v\| = \sqrt{x_1^2 + x_2^2 + \dots + x_n^2}$$

**Implicação:** Rankings tendem a ser idênticos!

### Quando as métricas podem divergir?

1. **Vetores não-normalizados:**
   - Se usar modelos locais sem normalização
   - Cosseno ignora magnitude, L2 não

2. **Documentos muito curtos vs. muito longos:**
   - L2 pode favorecer textos de tamanho similar à query
   - Cosseno é invariante ao tamanho

3. **Multimodalidade (texto + imagens):**
   - Diferentes modalidades podem ter escalas diferentes
   - Cosseno é mais robusto

### Qual usar em produção?

**Recomendação geral:**
- 🟢 **Similaridade de Cosseno** - Padrão da indústria para embeddings de texto
- 🔵 **L2** - Também funciona bem com embeddings OpenAI

**Decisão prática:**
- Para OpenAI/Google embeddings → **Cosseno** (convenção)
- Para embeddings customizados → Testar ambos e comparar
- Para multimodal → **Cosseno** (mais robusto)

## 📈 Análise Detalhada: Todos os Documentos com Scores

Vamos ver os scores de **todos** os 5 documentos para entender melhor como cada métrica ranqueia os resultados.

In [8]:
import pandas as pd
import numpy as np

print("🔬 ANÁLISE COMPLETA - TODOS OS DOCUMENTOS")
print("="*70)

# Buscar TODOS os documentos (k=5)
resultados_l2_todos = vector_store_l2.similarity_search_with_score(query, k=5)
resultados_cosine_todos = vector_store_cosine.similarity_search_with_score(query, k=5)

# Criar tabela comparativa
dados_comparacao = []

for i in range(5):
    doc_l2, score_l2 = resultados_l2_todos[i]
    doc_cosine, score_cosine = resultados_cosine_todos[i]
    
    # Calcular ângulo do cosseno
    similarity_clamped = np.clip(score_cosine, -1.0, 1.0)
    angulo_radianos = np.arccos(similarity_clamped)
    angulo_graus = np.degrees(angulo_radianos)
    
    # Extrair categoria do documento
    texto = doc_l2.page_content
    if "iPhone" in texto or "RTX" in texto or "placa" in texto:
        categoria = "🖥️ Tech"
    elif "bolo" in texto or "lasanha" in texto or "Receita" in texto:
        categoria = "🍰 Culinária"
    else:
        categoria = "⚽ Esporte"
    
    dados_comparacao.append({
        'Rank L2': i + 1,
        'Rank Cosseno': i + 1,
        'Categoria': categoria,
        'Documento': texto[:50] + "..." if len(texto) > 50 else texto,
        'Score L2': f"{score_l2:.4f}",
        'Score Cosseno': f"{score_cosine:.4f}",
        'Ângulo (°)': f"{angulo_graus:.2f}"
    })

# Criar DataFrame
df_comparacao = pd.DataFrame(dados_comparacao)

print("\n📊 Tabela Comparativa de Rankings:\n")
print(df_comparacao.to_string(index=False))

print("\n" + "="*70)
print("\n💡 Interpretação:")
print("  • L2: Scores MENORES = mais similar (distância)")
print("  • Cosseno: Scores MAIORES = mais similar (produto interno)")
print("  • Ângulo: MENOR ângulo = mais similar (0° = idêntico, 90° = perpendicular)")
print("  • ✅ Documentos de 'Tech' devem ter melhores scores e menores ângulos")
print("  • ❌ Documentos de 'Culinária' e 'Esporte' devem ter scores piores e ângulos maiores")
print("="*70)

🔬 ANÁLISE COMPLETA - TODOS OS DOCUMENTOS

📊 Tabela Comparativa de Rankings:

 Rank L2  Rank Cosseno   Categoria                                             Documento Score L2 Score Cosseno Ângulo (°)
       1             1     🖥️ Tech           A placa de vídeo RTX 4090 roda jogos em 4K.   1.4026        0.2983      72.65
       2             2     🖥️ Tech O novo iPhone 15 tem uma lente periscópica incríve...   1.4345        0.2828      73.57
       3             3 🍰 Culinária      Receita de lasanha à bolonhesa com muito queijo.   1.6202        0.1900      79.05
       4             4 🍰 Culinária     Para fazer um bolo macio, bata as claras em neve.   1.6491        0.1748      79.93
       5             5   ⚽ Esporte         O atacante chutou a bola no ângulo e foi gol.   1.7688        0.1156      83.36


💡 Interpretação:
  • L2: Scores MENORES = mais similar (distância)
  • Cosseno: Scores MAIORES = mais similar (produto interno)
  • Ângulo: MENOR ângulo = mais similar (0° = idêntico,

## 📋 Resumo Executivo: L2 vs Cosseno vs Ângulo

### Principais Diferenças

| Aspecto | Distância L2 (Euclidiana) | Similaridade de Cosseno | Ângulo do Cosseno |
|---------|---------------------------|------------------------|-------------------|
| **Fórmula** | $\sqrt{\sum(v_1 - v_2)^2}$ | $\frac{v_1 \cdot v_2}{\|v_1\| \|v_2\|}$ | $\arccos(\text{cosseno})$ |
| **O que mede** | Distância no espaço | Produto escalar normalizado | Orientação angular |
| **Escala** | 0 a ∞ | -1 a 1 | 0° a 180° |
| **Interpretação** | Menor = mais similar | Maior = mais similar | Menor = mais similar |
| **Valor ideal** | 0 (idêntico) | 1 (idêntico) | 0° (idêntico) |
| **Sensível à magnitude** | ✅ Sim | ❌ Não |  |
| **Normalização** | Não necessária | Automática (FAISS) |  |
| **Uso em NLP** | 🟡 Comum | 🟢 Padrão da indústria |  |
| **OpenAI/Google** | 🟢 Funciona bem | 🟢 Recomendado |  |
| **Performance** | ⚡ Muito rápida | ⚡ Muito rápida |  |

### 📐 Interpretação do Ângulo

**Faixas de Similaridade:**

| Ângulo | Cosseno aproximado | Classificação | Exemplo |
|--------|-------------------|---------------|---------|
| **0° - 30°** | 0.87 - 1.00 | 🟢 Muito similar | Query vs documento relevante |
| **30° - 60°** | 0.50 - 0.87 | 🟡 Moderadamente similar | Tópicos relacionados |
| **60° - 90°** | 0.00 - 0.50 | 🔴 Pouco similar | Tópicos diferentes |
| **90°** | 0.00 | ⚫ Perpendicular | Sem relação semântica |
| **90° - 180°** | -1.00 - 0.00 | 🟣 Oposto | Significados contrários |

**Vantagens do Ângulo:**
- ✅ **Intuição geométrica:** Fácil visualizar mentalmente
- ✅ **Escala familiar:** Graus são mais intuitivos que valores -1 a 1
- ✅ **Interpretação direta:** "20° = muito próximo, 80° = muito distante"
- ✅ **Educacional:** Ajuda a entender embeddings geometricamente

**Conversão:**
```python
# De cosseno para ângulo
import numpy as np
angulo_rad = np.arccos(cosseno)
angulo_graus = np.degrees(angulo_rad)

# De ângulo para cosseno
cosseno = np.cos(np.radians(angulo_graus))
```

### Resultado Esperado

**Para embeddings OpenAI (normalizados):**
- ✅ **Rankings idênticos** ou muito similares em todas as 3 métricas
- ✅ L2 e Cosseno são matematicamente relacionados para vetores unitários
- ✅ Ângulo fornece interpretação mais intuitiva do cosseno
- ✅ Todas identificam corretamente documentos relevantes

**Relação matemática (vetores normalizados):**
$$d_{L2}^2 = 2(1 - \cos\theta) = 2(1 - \text{cosseno})$$

**Quando divergem:**
- Vetores não-normalizados
- Diferentes magnitudes (textos curtos vs longos)
- Embeddings customizados sem normalização

### Recomendação Final

🏆 **Use Similaridade de Cosseno** como padrão para embeddings de texto (convenção da área)

📐 **Exiba o Ângulo** para interpretação humana e visualização

🔧 **Use L2** se precisar considerar magnitude dos vetores ou para compatibilidade com sistemas legados

### 💡 Dica Prática

Para apresentações e relatórios:
- **Técnico:** Mostre scores de cosseno (valores numéricos precisos)
- **Gestão/Stakeholders:** Mostre ângulos em graus (mais intuitivo)
- **Desenvolvedor:** Use L2 se pipeline já estiver configurado assim


### 🧪 Experimentos para Tentar

#### Experimento 1: Testar outras queries com ambas métricas
```python
# Query específica
pergunta_teste = "receitas de massas italianas"

print("🔵 L2:")
resultados_l2 = vector_store_l2.similarity_search(pergunta_teste, k=2)
for doc in resultados_l2:
    print(f"  • {doc.page_content}")

print("\n🟢 Cosseno:")
resultados_cos = vector_store_cosine.similarity_search(pergunta_teste, k=2)
for doc in resultados_cos:
    print(f"  • {doc.page_content}")

# Deve retornar: lasanha (e talvez bolo como 2º)

# Query ambígua
pergunta_teste = "como melhorar performance"
# Vai retornar hardware ou esporte? 🤔
# Compare os resultados das duas métricas!

# Query fora do domínio
pergunta_teste = "viagens para a Europa"
# Vai retornar o que está "menos distante" (mas nada relevante)
# As métricas concordam sobre qual é o "menos pior"?
```


In [9]:
# Código aqui


#### Experimento 2: Ajustar k (Top-K)
```python
# Ver todos os resultados
resultados = vector_store.similarity_search(pergunta, k=5)
# Agora você vê TODOS os 5 documentos ordenados por relevância
```


In [10]:
# Código aqui


#### Experimento 3: Comparar scores com threshold (limiar de relevância)
```python
# Ver os scores de similaridade e definir um threshold
pergunta_teste = "Quero sugestões de hardware para computador ou celular"

print("🔵 L2 - Análise de Threshold:")
resultados_l2 = vector_store_l2.similarity_search_with_score(pergunta_teste, k=5)
for doc, score in resultados_l2:
    relevante = "✅ RELEVANTE" if score < 1.0 else "❌ IRRELEVANTE"
    print(f"  Score: {score:.4f} {relevante} | {doc.page_content[:50]}")

print("\n🟢 Cosseno - Análise de Threshold:")
resultados_cos = vector_store_cosine.similarity_search_with_score(pergunta_teste, k=5)
for doc, score in resultados_cos:
    relevante = "✅ RELEVANTE" if score > 0.7 else "❌ IRRELEVANTE"
    print(f"  Score: {score:.4f} {relevante} | {doc.page_content[:50]}")

# Típico para L2: <0.8 = muito relevante, >2.0 = pouco relevante
# Típico para Cosseno: >0.8 = muito relevante, <0.5 = pouco relevante
```


In [11]:
# Código aqui


#### Experimento 4: Comparar com Google Embeddings
```python
# Descomente as linhas de GoogleGenerativeAIEmbeddings
# Crie um novo vector_store com Google
# Compare os resultados!

# Perguntas para reflexão:
# - Os resultados são iguais?
# - Um modelo é mais rápido?
# - Qual você prefere?
```



### 💡 Conceitos-chave para lembrar

1. **Embedding transforma texto em vetor** via API na nuvem
2. **OpenAI = qualidade máxima** mas paga por uso
3. **FAISS armazena vetores** e faz buscas instantâneas localmente
4. **Duas métricas principais:**
   - **L2 (Euclidiana):** Distância no espaço (menor = melhor)
   - **Cosseno:** Ângulo entre vetores (maior = melhor)
5. **Para embeddings OpenAI, ambas funcionam bem** (vetores normalizados)
6. **k define quantos resultados** você quer ver
7. **Scores ajudam a definir thresholds** de relevância

### 🎯 Quando usar cada métrica?

| Cenário | Métrica Recomendada | Motivo |
|---------|-------------------|---------|
| **Embeddings OpenAI/Google** | 🟢 Cosseno | Padrão da indústria, vetores normalizados |
| **Embeddings locais (Ollama)** | 🔵 L2 ou 🟢 Cosseno | Testar ambos e comparar |
| **Textos de tamanhos muito variados** | 🟢 Cosseno | Invariante à magnitude |
| **Dados não-normalizados** | 🔵 L2 | Considera magnitude |
| **Multimodal (texto + imagem)** | 🟢 Cosseno | Mais robusto entre modalidades |
| **Quando em dúvida** | 🟢 Cosseno | Convenção para NLP |

### 📐 Relação matemática entre L2 e Cosseno

Para vetores normalizados (||v|| = 1), como os da OpenAI:

$$d_{L2}^2(v_1, v_2) = 2(1 - \cos(v_1, v_2))$$

**Implicação:** Quando vetores são normalizados, L2 e Cosseno produzem o **mesmo ranking**!

A diferença é apenas na escala dos scores.


### 💰 Gestão de Custos e Performance

**Boas práticas:**
- ✅ Cache embeddings já calculados (não recalcular)
- ✅ Use batch processing para múltiplos textos
- ✅ Monitore uso com `openai.api_requestor`
- ✅ Considere rate limits (3,000 RPM tier gratuito)
- ✅ Reutilize índices FAISS (salve em disco se necessário)

**Exemplo de custo real:**
```text
Aplicação com 10k documentos:
- Indexação inicial: 10k docs × 50 tokens = 500k tokens
- Custo: 500k / 1M × $0.02 = $0.01 (um centavo!)
- Queries: praticamente grátis (<$0.001 por 1000 queries)
  (vetores já estão no cache, busca é local no FAISS)
```

**Performance das métricas:**

| Métrica | Velocidade | Memória | Precisão |
|---------|-----------|---------|----------|
| **L2** | ⚡⚡⚡ Muito rápida | 💾 Baixa | ✅ Excelente |
| **Cosseno (IP)** | ⚡⚡⚡ Muito rápida | 💾 Baixa | ✅ Excelente |

**Ambas são igualmente rápidas no FAISS!** A escolha é mais sobre interpretação dos scores e convenção da área.

### ⚡ Otimizações Avançadas

1. **Salvar índice FAISS em disco:**
```python
# Salvar
vector_store_cosine.save_local("meu_indice_faiss")

# Carregar depois (sem chamar API novamente!)
vector_store = FAISS.load_local("meu_indice_faiss", embeddings)
```

2. **Criar índices mais eficientes para grandes datasets:**
```python
# IVF (Inverted File) para milhões de vetores
from langchain_community.vectorstores import FAISS
import faiss

# Usar IndexIVFFlat ao invés de IndexFlat
# Muito mais rápido para >100k documentos
```

3. **Filtrar por metadata antes da busca:**
```python
# Adicionar metadata aos documentos
metadatas = [{"categoria": "tech"}, {"categoria": "food"}, ...]
vector_store = FAISS.from_texts(textos, embeddings, metadatas=metadatas)

# Buscar apenas em uma categoria
resultados = vector_store.similarity_search(
    pergunta, 
    k=2,
    filter={"categoria": "tech"}
)
```



### 🎓 Em resumo

Você acabou de construir um sistema de busca semântica com **qualidade OpenAI** e aprendeu a comparar diferentes métricas de similaridade! 🎉

**O que você dominou:**
- ✅ Embeddings via API em nuvem (OpenAI)
- ✅ Indexação vetorial com FAISS
- ✅ **Distância L2 (Euclidiana)** - mede distância no espaço
- ✅ **Similaridade de Cosseno** - mede ângulo entre vetores
- ✅ Comparação de métricas e interpretação de scores
- ✅ Visualização e análise de resultados
- ✅ Otimizações e boas práticas

Este é o mesmo princípio usado em:
- 🔍 **Buscadores avançados** (Elasticsearch, Algolia)
- 🤖 **Chatbots empresariais** (RAG - Retrieval Augmented Generation)
- 📚 **Sistemas de recomendação** (Netflix, Spotify, Amazon)
- 🔬 **Ferramentas de pesquisa acadêmica** (Semantic Scholar)
- 💼 **Busca semântica corporativa** (Confluence, Notion)


### 📚 Recursos Adicionais

**Documentação:**
- [OpenAI Embeddings Guide](https://platform.openai.com/docs/guides/embeddings)
- [FAISS Documentation](https://github.com/facebookresearch/faiss/wiki)
- [LangChain Vector Stores](https://python.langchain.com/docs/modules/data_connection/vectorstores/)

**Benchmarks:**
- [MTEB Leaderboard](https://huggingface.co/spaces/mteb/leaderboard) - Compare modelos de embedding

### ✅ Checklist de Aprendizado

Marque os conceitos que você compreendeu:

**Fundamentos:**
- [ ] O que são embeddings e como são gerados via API
- [ ] Diferença entre busca tradicional e semântica
- [ ] Como FAISS armazena e busca vetores

**Métricas de Similaridade:**
- [ ] O que é Distância L2 (Euclidiana) e como interpretar
- [ ] O que é Similaridade de Cosseno e como interpretar
- [ ] O que é o Ângulo do Cosseno e como interpretar
- [ ] Diferenças entre as três métricas
- [ ] Quando usar cada uma
- [ ] Relação matemática entre cosseno e ângulo (θ = arccos(similarity))

**Prática:**
- [ ] Executei comparações com ambas métricas
- [ ] Interpretei os scores corretamente (L2, Cosseno e Ângulo)
- [ ] Visualizei os resultados em gráficos
- [ ] Testei queries diferentes
- [ ] Entendi que ângulos menores = maior similaridade

**Conceitos Avançados:**
- [ ] Relação matemática entre L2 e Cosseno (vetores normalizados)
- [ ] Por que OpenAI normaliza embeddings
- [ ] Como definir thresholds de relevância (em todas as métricas)
- [ ] Estratégias de otimização (cache, batch processing)
- [ ] Conversão entre cosseno e ângulo (arccos e cos)

**Interpretação Geométrica:**
- [ ] Visualizei vetores como setas no espaço
- [ ] Entendi que 0° = idêntico, 90° = perpendicular
- [ ] Compreendi que ângulo é mais intuitivo para humanos
- [ ] Aprendi faixas de interpretação (0-30° muito similar, 60-90° pouco similar)

**Se você marcou todos:** Parabéns! Você está pronto para o próximo lab! 🚀